In [12]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime

def classify_and_save_image(image):
    # Create output folders if they don't exist
    categories = ['left', 'slight_left', 'center', 'slight_right', 'right']
    for category in categories:
        os.makedirs(os.path.join(category), exist_ok=True)

    # Get image dimensions
    height, width = image.shape
    section_width = width // 5

    # Count white pixels in each section
    white_counts = [np.sum(image[:, i * section_width: (i + 1) * section_width] == 255) for i in range(5)]

    # Determine the section with the most white pixels
    max_index = np.argmax(white_counts)
    category = categories[max_index]

    # Save the image with a unique name
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S%f')
    output_path = os.path.join(category, f"{timestamp}.png")
    cv2.imwrite(output_path, image)

    print(f"Image saved to {output_path} under category '{category}'")


# Example usage
# classify_and_save_image('path_to_image.png', 'output_folder')
def store_image(folder, image):
    """Stores an image in the specified folder with a timestamp-based filename."""
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist

    # Generate a unique filename using date and time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")+f"_{datetime.now().microsecond // 1000}"
    filename = f"{folder}/image_{timestamp}.png"

    # Save image
    cv2.imwrite(filename, image)
    
def cut_images(folder, image):
    """Detects the centerline of a white line in a black background and returns (x, y) points."""
    height, width = image.shape[:2]
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    row_index = 0
    # Region of interest (ROI) is the top half of the image as the robot only moves forward
    for i in range(4):
        roi = gray[row_index:row_index+50, :] 
        row_index += 50
        # Threshold to isolate the white line
        _, binary = cv2.threshold(roi, 200, 255, cv2.THRESH_BINARY)
        classify_and_save_image(binary)
        classify_and_save_image(cv2.flip(binary, 1))

def calculate_angle(centerline):
    """Calculates the angle of the white line with respect to the vertical axis."""
    
    if len(centerline) < 2:
        return None  # Not enough points to calculate angle

    # Fit a line using linear regression
    x_coords = np.array([p[0] for p in centerline])
    y_coords = np.array([p[1] for p in centerline])
    
    # Fit a line: y = mx + b
    m, b = np.polyfit(x_coords, y_coords, 1)  # Slope and intercept

    # Calculate angle from vertical axis
    angle_rad = np.arctan(m)
    angle_deg = np.degrees(angle_rad)

    return angle_deg

def process_images(folder, processed_folder):
    """Processes all images in a folder and saves centerline points + angle."""
    
    if not os.path.exists(folder):
        print("Folder does not exist.")
        return

    image_files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print("No images found.")
        return

    for filename in image_files:
        image_path = os.path.join(folder, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Failed to read {filename}")
            continue

        cut_images(processed_folder, image)


In [14]:
process_images("images", "processed")   

Image saved to center\20250319_204251678496.png under category 'center'
Image saved to center\20250319_204251681026.png under category 'center'
Image saved to center\20250319_204251683044.png under category 'center'
Image saved to center\20250319_204251685042.png under category 'center'
Image saved to center\20250319_204251686043.png under category 'center'
Image saved to center\20250319_204251688041.png under category 'center'
Image saved to center\20250319_204251689042.png under category 'center'
Image saved to center\20250319_204251690042.png under category 'center'
Image saved to center\20250319_204251693393.png under category 'center'
Image saved to center\20250319_204251695022.png under category 'center'
Image saved to center\20250319_204251695620.png under category 'center'
Image saved to center\20250319_204251695620.png under category 'center'
Image saved to center\20250319_204251698188.png under category 'center'
Image saved to center\20250319_204251699197.png under category '

In [ ]:
import os
import shutil
import random

def move_images(train_dir: str, valid_dir: str, num_images: int = 100):
    categories = ['left', 'slight_left', 'center', 'slight_right', 'right','crossing']
    for category in categories:
        train_category_path = os.path.join(train_dir, category)
        valid_category_path = os.path.join(valid_dir, category)
        os.makedirs(valid_category_path, exist_ok=True)

        # List all images in the category folder
        images = [f for f in os.listdir(train_category_path) if os.path.isfile(os.path.join(train_category_path, f))]

        # Randomly select images
        selected_images = random.sample(images, min(num_images, len(images)))

        for image in selected_images:
            src_path = os.path.join(train_category_path, image)
            dest_path = os.path.join(valid_category_path, image)

            # Move image to validation folder
            shutil.move(src_path, dest_path)

        print(f"Moved {len(selected_images)} images from {train_category_path} to {valid_category_path}")


# Example usage
#move_images('dataset/train', 'dataset/valid')


Moved 100 images from dataset/train\crossing to dataset/valid\crossing


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LineFollower(nn.Module):
    def __init__(self, num_classes):  
        super(LineFollower, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=50, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=50, out_channels=30, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=5, stride=5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(30 * 5 * 32, 128)  # Assuming input size is (50, 320)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        # Flatten the output
        x = x.view(x.size(0), -1)
        
        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))
        
        # Output layer
        x = self.fc2(x)
        
        return x
    
# Create a model
model = LineFollower(num_classes=6) 
print(model)

LineFollower(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(50, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4800, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=6, bias=True)
)


In [26]:
# LOAD DATA

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define directories for train and validation
train_dir = "dataset/train"
valid_dir = "dataset/valid"

# Define image transformations
transform = transforms.Compose([
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Grayscale(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pre-trained models
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

# Check the labels
print("Classes:", train_dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(valid_dataset))

# Checking a batch of images
images, labels = next(iter(train_loader))
print("Image batch shape:", images.shape)
print("Labels batch shape:", labels.shape)

Classes: ['center', 'crossing', 'left', 'right', 'slight_left', 'slight_right']
Number of training samples: 32267
Number of validation samples: 600
Image batch shape: torch.Size([32, 1, 50, 320])
Labels batch shape: torch.Size([32])


In [33]:
# TRAINING 
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# Select the available device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
learning_rate = 0.001
num_epochs = 1

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        # Move data to GPU if available
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch}/{num_epochs}]: loss = {loss.item()}")

100%|██████████| 1/1 [05:05<00:00, 305.29s/it]

Epoch [0/1]: loss = 0.003938284236937761


In [34]:
# Accuracy on the test set
correct = 0
total = len(valid_dataset)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(valid_loader):
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        x = images  
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

accuracy = correct/total
print('Test accuracy: {}'.format(accuracy))

100%|██████████| 19/19 [00:01<00:00, 13.56it/s]

Test accuracy: 0.9883333444595337


In [35]:
# Compare and save the best model

import os
import torch
import re

def save_best_model(model, new_accuracy, directory=".", model_name="best_model"):
    """
    Saves the model if its accuracy is higher than the previous best one.
    
    Args:
        model (torch.nn.Module): The PyTorch model to save.
        new_accuracy (float): The new model's accuracy to compare.
        directory (str): Directory where the model files are saved.
        model_name (str): Base name of the model file.
        
    Returns:
        bool: True if the new model was saved, False otherwise.
    """
    best_accuracy = 0.0
    best_model_file = None

    # Look for the best model file in the directory
    for filename in os.listdir(directory):
        match = re.match(rf"{model_name}_(\d+.\d+).pt", filename)
        if match:
            saved_accuracy = float(match.group(1))
            if saved_accuracy > best_accuracy:
                best_accuracy = saved_accuracy
                best_model_file = filename

    # Compare accuracies
    if new_accuracy > best_accuracy:
        # Save new model
        new_model_file = f"{model_name}_{new_accuracy:.2f}.pt"
        torch.save(model.state_dict(), os.path.join(directory, new_model_file))
        print(f"New best model saved: {new_model_file} (Accuracy: {new_accuracy:.2f}%)")
        return True
    else:
        print(f"No improvement. Current best model: {best_model_file} (Accuracy: {best_accuracy:.2f}%)")
        return False
    
save_best_model(model, accuracy)

No improvement. Current best model: best_model_0.99.pt (Accuracy: 0.99%)


False

In [6]:
# TEST MODEL ON A SINGLE IMAGE

from PIL import Image
from CNNmodel import LineFollower
import torch
import cv2
from torchvision import transforms

transform = transforms.Compose([
        transforms.ToTensor(),           # Convert images to PyTorch tensors
        transforms.Grayscale(),
    ])

model = LineFollower(num_classes=6)
model.load_state_dict(torch.load('best_model_0.99.pt'))
categories = ['center', 'crossing', 'left', 'right', 'slight_left', 'slight_right']

image = cv2.imread("test_img.png")
# Apply the transform
pil_image = Image.fromarray(image)
img = transform(pil_image)

# Now you can pass it to the model
prob = model(img.unsqueeze(0))  # Add a batch dimension if the model expects it
print(prob)
predictions = torch.argmax(prob, dim=1)
predictions

tensor([[-3.3527,  0.5508, -7.6807, -3.6213, -5.5003, -1.4040]],
       grad_fn=<AddmmBackward0>)


tensor([1])